In [121]:
!pip install --upgrade pip
!pip install pyarrow
!pip install pandas
!pip install duckdb
!pip install hnswlib

In [122]:
import pyarrow.parquet as pq
import numpy as np
import pandas as pd
import pyarrow as pa
import duckdb
import hnswlib

In [123]:
test_table = pq.read_table('test_data_to_record.parquet')
panda_test_table = test_table.to_pandas()
# print(panda_test_table.head())

train_table = pq.read_table('train_data_to_record.parquet')
panda_train_table = train_table.to_pandas()
print(panda_train_table.head())


                                      embedding_data  \
0  [-19.19756317138672, -18.10773468017578, -22.2...   
1  [6.993745803833008, -16.4917049407959, -12.157...   
2  [-13.4398193359375, -6.445910930633545, -7.608...   
3  [-15.908609390258789, 7.963416576385498, -12.5...   
4  [-16.739404678344727, -16.108036041259766, -20...   

                resource_uri                                 metadata  \
0  train-images-idx3-ubyte-0        {'location': 'DC', 'quality': 51}   
1  train-images-idx3-ubyte-1    {'location': 'Dallas', 'quality': 25}   
2  train-images-idx3-ubyte-2  {'location': 'New York', 'quality': 23}   
3  train-images-idx3-ubyte-3   {'location': 'Chicago', 'quality': 18}   
4  train-images-idx3-ubyte-4   {'location': 'Chicago', 'quality': 87}   

                                   infer  \
0  {'annotations': [{'category_id': 5}]}   
1  {'annotations': [{'category_id': 0}]}   
2  {'annotations': [{'category_id': 4}]}   
3  {'annotations': [{'category_id': 1}]}   
4  {

In [124]:
# print(duckdb.query('''
# SELECT COUNT(*)
# FROM 'test_data_to_record.parquet'
# WHERE metadata.location ILIKE '%San Francisco%';
# ''').fetchall())
print("Record in test")
print(duckdb.query('''
SELECT COUNT(*)
FROM 'test_data_to_record.parquet'
;
''').fetchall())
print("Record in train")
print(duckdb.query('''
SELECT COUNT(*)
FROM 'train_data_to_record.parquet'
;
''').fetchall())

Record in test
[(10000,)]
Record in train
[(60000,)]


In [125]:
import hnswlib
import numpy as np

#dim = 16
# num_elements = 10000

# Generating sample data
# data = np.float32(np.random.random((num_elements, dim)))

# We split the data in two batches:
data1 = data[:num_elements // 2]
data2 = data[num_elements // 2:]

data1 = panda_test_table['embedding_data'].to_numpy().tolist()
data2 = panda_train_table['embedding_data'].to_numpy().tolist()
dim = len(data1[0])
num_elements = len(data1) + len(data2)
# print("dimensionality is:", dim)
# print("total number of elements is:", num_elements)
# print("max elements", num_elements//2)

concatted_data = data1 + data2
# print("concatted_data", len(concatted_data))

# Declaring index
p = hnswlib.Index(space='l2', dim=dim)  # possible options are l2, cosine or ip

# Initing index
# max_elements - the maximum number of elements (capacity). Will throw an exception if exceeded
# during insertion of an element.
# The capacity can be increased by saving/loading the index, see below.
#
# ef_construction - controls index search speed/build speed tradeoff
#
# M - is tightly connected with internal dimensionality of the data. Strongly affects the memory consumption (~M)
# Higher M leads to higher accuracy/run_time at fixed ef/efConstruction

p.init_index(max_elements=len(data1), ef_construction=100, M=16)

# Controlling the recall by setting ef:
# higher ef leads to better accuracy, but slower search
p.set_ef(10)

# Set number of threads used during batch search/construction
# By default using all available cores
p.set_num_threads(4)

# print(data1[0])

# print("Adding first batch of %d elements" % (len(data1)))
p.add_items(data1)

# Query the elements for themselves and measure recall:
labels, distances = p.knn_query(data1, k=1)
print(labels)

print(distances)
# print("Recall for the first batch:", np.mean(labels.reshape(-1) == np.arange(len(data1))), "\n")

# Serializing and deleting the index:
index_path='first_half.bin'
# print("Saving index to '%s'" % index_path)
p.save_index(index_path)
del p

# Reiniting, loading the index
p = hnswlib.Index(space='l2', dim=dim)  # the space can be changed - keeps the data, alters the distance function.

# print("\nLoading index from 'first_half.bin'\n")

# Increase the total capacity (max_elements), so that it will handle the new data
p.load_index("first_half.bin", max_elements = num_elements)

# print("Adding the second batch of %d elements" % (len(data2)))
p.add_items(data2)

# Query the elements for themselves and measure recall:
labels, distances = p.knn_query(concatted_data, k=1)
# print("Recall for two batches:", np.mean(labels.reshape(-1) == np.arange(len(concatted_data))), "\n")

[[   0]
 [   1]
 [   2]
 ...
 [9997]
 [9998]
 [9999]]
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [126]:
def unpack_annotations(embeddings):
    # Get and unpack inference data
    annotations = [
        embedding['infer']["annotations"]
        for embedding in embeddings
    ]  # Unpack JSON
    annotations = [
        annotation for annotation_list in annotations for annotation in annotation_list
    ]  # Flatten the list

    # categories_by_uid = {
    #     annotation["id"]: annotation["category_id"] for annotation in annotations
    # }
    # print(categories_by_uid)

    # # Unpack embedding data
    embeddings = [embedding["embedding_data"] for embedding in embeddings]

    embedding_vectors_by_category = {}
    for embedding_annotation_pair in zip(embeddings, annotations):
        data = np.array(embedding_annotation_pair[0])
        category = embedding_annotation_pair[1]['category_id'] #categories_by_uid[embedding["target"]]
        if category in embedding_vectors_by_category.keys():
            embedding_vectors_by_category[category] = np.append(
                embedding_vectors_by_category[category], data[np.newaxis, :], axis=0
            )
        else:
            embedding_vectors_by_category[category] = data[np.newaxis, :]

    return embedding_vectors_by_category

# Get the training embeddings. This is the set of embeddings belonging to datapoints of the training dataset, and to the first created embedding set.
training_embedding_vectors_by_category = unpack_annotations(panda_train_table.to_dict('records'))




In [127]:
inv_covs_by_category = {}
means_by_category = {}
for category, embeddings in training_embedding_vectors_by_category.items():
    print(f"Computing mean and covariance for label categry {category}")

    # Compute the mean and inverse covariance for computing MHB distance
    print(f"category: {category} samples: {embeddings.shape[0]}")
    if embeddings.shape[0] < (embeddings.shape[1] + 1):
        print(f"not enough samples for stable covariance in category {category}")
        continue
    cov = np.cov(embeddings.transpose())
    try:
        inv_cov = np.linalg.inv(cov)
    except np.linalg.LinAlgError as err:
        print(f"covariance for category {category} is singular")
        continue
    mean = np.mean(embeddings, axis=0)
    inv_covs_by_category[category] = inv_cov
    means_by_category[category] = mean

target_datapoints = panda_test_table.to_dict('records')

# Process each datapoint's inferences individually. This is going to be very slow.
# This is because there is no way to grab the corresponding metadata off the datapoint.
# We could instead put it on the embedding directly ?
inference_metadata = {}
for idx, datapoint in enumerate(target_datapoints):
    inferences = datapoint['infer']["annotations"]
    embeddings = [datapoint["embedding_data"]]

    for i in range(len(inferences)):
        emb_data = embeddings[i]
        category = inferences[i]["category_id"]
        if not category in inv_covs_by_category.keys():
            continue
        mean = means_by_category[category]
        inv_cov = inv_covs_by_category[category]
        delta = np.array(emb_data) - mean
        squared_mhb = np.sum((delta * np.matmul(inv_cov, delta)), axis=0)
        if squared_mhb < 0:
            print(f"squared distance for category {category} is negative")
            continue
        distance = np.sqrt(squared_mhb)
        inference_metadata[idx] = distance

# dict to json
import json
print(json.dumps(inference_metadata))

Computing mean and covariance for label categry 5
category: 5 samples: 5419
Computing mean and covariance for label categry 0
category: 0 samples: 5932
Computing mean and covariance for label categry 4
category: 4 samples: 5845
Computing mean and covariance for label categry 1
category: 1 samples: 6741
Computing mean and covariance for label categry 9
category: 9 samples: 5937
Computing mean and covariance for label categry 2
category: 2 samples: 5953
Computing mean and covariance for label categry 3
category: 3 samples: 6127
Computing mean and covariance for label categry 6
category: 6 samples: 5919
Computing mean and covariance for label categry 7
category: 7 samples: 6279
Computing mean and covariance for label categry 8
category: 8 samples: 5848
{"0": 1.9063181237952946, "1": 3.2871343891627207, "2": 1.7366561686540263, "3": 2.179490844144519, "4": 2.195223800144155, "5": 1.8615185941496668, "6": 3.209138441323173, "7": 3.2440197128298665, "8": 4.119089078027721, "9": 2.92826583411